In [1]:
from langchain_mcp_tools import convert_mcp_to_langchain_tools
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
import asyncio

In [2]:
async def main():
    # NOTE: No need to instantiate fastmcp.Client here
    # convert_mcp_to_langchain_tools will handle servers for you
    tools, cleanup = await convert_mcp_to_langchain_tools({
        "math": {
            "url": "http://localhost:8000/mcp/",
            "transport": "streamable-http",
        }
    })

    llm   = ChatOllama(model="llama3.2:3b", temperature=0)
    agent = create_react_agent(llm, tools)

    # Example queries
    for query in ["What is 7 + 13?", "Compute 6 × 9."]:
        result = await agent.ainvoke(
            {"messages": [{"role": "user", "content": query}]}
        )
        last   = result["messages"][-1]
        answer = last.get("content") if isinstance(last, dict) else last.content
        print(f"Query: {query}\nAnswer: {answer}\n")

    # Clean up MCP server sessions
    await cleanup()

In [3]:
await main()

[INFO] MCP server "math": initializing with: {'url': 'http://localhost:8000/mcp/', 'transport': 'streamable-http'}
[INFO] MCP server "math": Pre-validating authentication
[INFO] MCP server "math": Authentication validation passed: 200
[INFO] MCP server "math": testing Streamable HTTP support for http://localhost:8000/mcp/
[INFO] MCP server "math": detected Streamable HTTP transport support
[INFO] Received session ID: b7bc0f5cfd964612bde1d4c665ad7607
[INFO] Negotiated protocol version: 2025-06-18
[INFO] MCP server "math": connected
[INFO] MCP server "math": 2 tool(s) available:
[INFO] - add
[INFO] - multiply
[INFO] MCP servers initialized: 2 tool(s) available in total
[INFO] MCP tool "math"/"add" received input: {'a': 7, 'b': 13}
[INFO] MCP tool "math"/"add" received result (size: 2)


Query: What is 7 + 13?
Answer: The answer to the original question, "What is 7 + 13?", is 20.



[INFO] MCP tool "math"/"multiply" received input: {'a': 6, 'b': 9}
[INFO] MCP tool "math"/"multiply" received result (size: 2)
[INFO] MCP server "math": session closed


Query: Compute 6 × 9.
Answer: The result of multiplying 6 and 9 is 54.

